# Web Scraping Basketball Reference Data

I will be using the website Basketball-reference.com in order to scrap data of the players from multiple different seasons. This website provides clean sport data, thus making it easy to scrape the data and create my own database from the data gathered on the website.

Now to import all the libraries we would need in order to begin web scarping the data from the website. 

In [2]:
# import needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

We will now create the function to gather data from the website. It will take in the year we want for that season. 

In [3]:
# function to gather data from the web parameter is the year
def scrape_nba_data(years):
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(years)

    try:
        html = urlopen(url)
        soup = BeautifulSoup(html, features="lxml")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers.remove('Rk')

    rows = soup.findAll('tr')[1:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')] 
                    for i in range(len(rows))]
    
    df = pd.DataFrame(rows_data, columns=headers)
    df.dropna(axis = 0, inplace=True)
    df = df.reset_index(drop=True)

    # Replace empty string with '0'
    df.replace('', '0', inplace=True)   

    # convert numeric columns to float
    numeric_columns = df.columns.drop(['Player', 'Pos', 'Tm'])
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    return df

Now that we created the function to be able to gather data let's test it out by looking at the 2020 - 2021 seaon.

In [4]:
# Test to make sure the function works
NBA_season_2020_2021 = scrape_nba_data(2021)
NBA_season_2020_2021

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,PF,21,MIA,61,4,12.1,2.0,3.7,0.544,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,Jaylen Adams,PG,24,MIL,7,0,2.6,0.1,1.1,0.125,...,0.000,0.0,0.4,0.4,0.3,0.0,0.0,0.0,0.1,0.3
2,Steven Adams,C,27,NOP,58,58,27.7,3.3,5.3,0.614,...,0.444,3.7,5.2,8.9,1.9,0.9,0.7,1.3,1.9,7.6
3,Bam Adebayo,C,23,MIA,64,64,33.5,7.1,12.5,0.570,...,0.799,2.2,6.7,9.0,5.4,1.2,1.0,2.6,2.3,18.7
4,LaMarcus Aldridge,C,35,TOT,26,23,25.9,5.4,11.4,0.473,...,0.872,0.7,3.8,4.5,1.9,0.4,1.1,1.0,1.8,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,0.462,...,0.833,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0
701,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,0.559,...,0.628,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1
702,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,0.438,...,0.886,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3
703,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,0.559,...,0.714,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4


In [9]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 15.3 MB 1.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Now to create a database in order to hold all the data we just got from Basketball Reference

In [5]:
import mysql.connector 

# Establish the connection
database = mysql.connector.connect(user='root', password='Xnomxslayer123!@#', host='localhost')

# Create a cursor object
cursor = database.cursor()

# Create the new database
cursor.execute("CREATE DATABASE IF NOT EXISTS nba_database")

# Use the new database
cursor.execute("USE nba_database")

Now we will create the table that we will to store the data.

In [6]:
cursor.execute("DROP TABLE IF EXISTS player_stats")

In [7]:
NBA_season_2020_2021.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [8]:
# Create a new table
cursor.execute("""CREATE TABLE player_stats
                (Player VARCHAR(50),
                Pos VARCHAR(10),
                Age INT,
                Tm VARCHAR(10),
                G INT,
                GS INT,
                MP FLOAT,
                FG FLOAT,
                FGA FLOAT,
                FG_PCT FLOAT,
                3P FLOAT,
                3PA FLOAT,
                3P_PCT FLOAT,
                2P FLOAT,
                2PA FLOAT,
                2P_PCT FLOAT,
                EFG_PCT FLOAT,
                FT FLOAT,
                FTA FLOAT,
                FT_PCT FLOAT,
                ORB FLOAT,
                DRB FLOAT,
                TRB FLOAT,
                AST FLOAT,
                STL FLOAT,
                BLK FLOAT,
                TOV FLOAT,
                PF FLOAT,
                PTS FLOAT)
               """)

Finally we can start adding values into our new table by creating a function that will take in a Pandas Dataframe and then it will insert everything intot the database.

In [10]:
# Create a function to insert data into the table with a parameter of a dataframe
def insert_data(nba_season):
    for index, row in nba_season.iterrows():
        query = """INSERT INTO player_stats(Player, Pos, Age, Tm, G, GS, MP, FG, FGA, FG_PCT, 3P, 3PA, 3P_PCT, 2P, 2PA, 2P_PCT, 
                                            EFG_PCT, FT, FTA, FT_PCT, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS)
                                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                                            %s, %s, %s, %s, %s, %s, %s, %s)"""
        data = (row['Player'], row['Pos'], row['Age'], row['Tm'], row['G'], row['GS'], row['MP'], row['FG'], row['FGA'], row['FG%'], 
                row['3P'], row['3PA'], row['3P%'], row['2P'], row['2PA'], row['2P%'], row['eFG%'], row['FT'], row['FTA'], row['FT%'], row['ORB'], row['DRB'], row['TRB'], row['AST'], row['STL'], row['BLK'], row['TOV'], row['PF'], row['PTS'])
        cursor.execute(query, data)

    # commit the transaction
    database.commit()
    # Consume any remaining unread results
    while cursor.nextset():
        pass


Now to insert the player stats from the the 2020-2021 season into the database.

In [11]:
insert_data(NBA_season_2020_2021)

Test to make sure we were able to correctly insert data into the database by looking at the first 5 rows.

In [12]:
query = """
    SELECT * 
    FROM player_stats
    LIMIT 5
    """

cursor.execute(query)

rows = cursor.fetchall()

for row in rows:
    print(row)

('Precious Achiuwa', 'PF', 21, 'MIA', 61, 4, 12.1, 2.0, 3.7, 0.544, 0.0, 0.0, 0.0, 2.0, 3.7, 0.546, 0.544, 0.9, 1.8, 0.509, 1.2, 2.2, 3.4, 0.5, 0.3, 0.5, 0.7, 1.5, 5.0)
('Jaylen Adams', 'PG', 24, 'MIL', 7, 0, 2.6, 0.1, 1.1, 0.125, 0.0, 0.3, 0.0, 0.1, 0.9, 0.167, 0.125, 0.0, 0.0, 0.0, 0.0, 0.4, 0.4, 0.3, 0.0, 0.0, 0.0, 0.1, 0.3)
('Steven Adams', 'C', 27, 'NOP', 58, 58, 27.7, 3.3, 5.3, 0.614, 0.0, 0.1, 0.0, 3.3, 5.3, 0.62, 0.614, 1.0, 2.3, 0.444, 3.7, 5.2, 8.9, 1.9, 0.9, 0.7, 1.3, 1.9, 7.6)
('Bam Adebayo', 'C', 23, 'MIA', 64, 64, 33.5, 7.1, 12.5, 0.57, 0.0, 0.1, 0.25, 7.1, 12.4, 0.573, 0.571, 4.4, 5.5, 0.799, 2.2, 6.7, 9.0, 5.4, 1.2, 1.0, 2.6, 2.3, 18.7)
('LaMarcus Aldridge', 'C', 35, 'TOT', 26, 23, 25.9, 5.4, 11.4, 0.473, 1.2, 3.1, 0.388, 4.2, 8.3, 0.505, 0.525, 1.6, 1.8, 0.872, 0.7, 3.8, 4.5, 1.9, 0.4, 1.1, 1.0, 1.8, 13.5)


In [13]:
query = """ 
        SELECT * 
        FROM player_stats
        LIMIT 5
        """

df = pd.read_sql_query(query, database)
print(df)

              Player Pos  Age   Tm   G  GS    MP   FG   FGA  FG_PCT  ...  \
0   Precious Achiuwa  PF   21  MIA  61   4  12.1  2.0   3.7   0.544  ...   
1       Jaylen Adams  PG   24  MIL   7   0   2.6  0.1   1.1   0.125  ...   
2       Steven Adams   C   27  NOP  58  58  27.7  3.3   5.3   0.614  ...   
3        Bam Adebayo   C   23  MIA  64  64  33.5  7.1  12.5   0.570  ...   
4  LaMarcus Aldridge   C   35  TOT  26  23  25.9  5.4  11.4   0.473  ...   

   FT_PCT  ORB  DRB  TRB  AST  STL  BLK  TOV   PF   PTS  
0   0.509  1.2  2.2  3.4  0.5  0.3  0.5  0.7  1.5   5.0  
1   0.000  0.0  0.4  0.4  0.3  0.0  0.0  0.0  0.1   0.3  
2   0.444  3.7  5.2  8.9  1.9  0.9  0.7  1.3  1.9   7.6  
3   0.799  2.2  6.7  9.0  5.4  1.2  1.0  2.6  2.3  18.7  
4   0.872  0.7  3.8  4.5  1.9  0.4  1.1  1.0  1.8  13.5  

[5 rows x 29 columns]


/var/folders/6w/gc_j5_3d0q3_r_2n1_t35yk00000gn/T/ipykernel_24946/1271988993.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, database)


Now that we can see that it worked we will now be able to gather more data from Basketball reference and store into the database for future analytical work on the data. 

In [ ]:
NBA_season_2020_2021

Let's gather data from the last 14 years of the NBA including the current season.

In [17]:
# Gathering data using the function we created to scrape and clean the data
NBA_season_2023_2024 = scrape_nba_data(2024)
NBA_season_2022_2023 = scrape_nba_data(2023)
NBA_season_2021_2022 = scrape_nba_data(2022)
NBA_season_2020_2021 = scrape_nba_data(2021)
NBA_season_2019_2020 = scrape_nba_data(2020)
NBA_season_2018_2019 = scrape_nba_data(2019)
NBA_season_2017_2018 = scrape_nba_data(2018)
NBA_season_2016_2017 = scrape_nba_data(2017)
NBA_season_2015_2016 = scrape_nba_data(2016)
NBA_season_2014_2015 = scrape_nba_data(2015)
NBA_season_2013_2014 = scrape_nba_data(2014)
NBA_season_2012_2013 = scrape_nba_data(2013)
NBA_season_2011_2012 = scrape_nba_data(2012)
NBA_season_2010_2011 = scrape_nba_data(2011)
NBA_season_2009_2010 = scrape_nba_data(2010)

Now we will insert the data we gathered from the past 14 years of NBA players statistics into the database we created earlier.

In [18]:
# here we are inserting the data into the database with the function we created
insert_data(NBA_season_2023_2024)
insert_data(NBA_season_2022_2023)
insert_data(NBA_season_2021_2022)
insert_data(NBA_season_2019_2020)
insert_data(NBA_season_2018_2019)
insert_data(NBA_season_2017_2018)
insert_data(NBA_season_2016_2017)
insert_data(NBA_season_2015_2016)
insert_data(NBA_season_2014_2015)
insert_data(NBA_season_2013_2014)
insert_data(NBA_season_2012_2013)
insert_data(NBA_season_2011_2012)
insert_data(NBA_season_2010_2011)
insert_data(NBA_season_2009_2010)


In [19]:
# We are going to look at the number of rows in the table to make sure the data was inserted.
query = """ 
        SELECT COUNT(*)
        FROM player_stats
        """

# Execute the query
cursor.execute(query)

# Fetch the results
rows = cursor.fetchone()

print("Number of rows in the table: ", rows[0])


Number of rows in the table:  9696


Now that we know we have the data stored in the database correctly we can now continue with making some visualizatons in order to get some insight from all the data we just got. After, we will move to tableau to create a dashboard to answer some analytical questions. 

The first question that I would like to be answered is if there is any correlation between a player's age and thier performance(like points per game, assists, rebounds, etc)?

First install and import the import libraries needed.

In [21]:
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

In [24]:
# Lets get the oldest player from last season 
old_player = NBA_season_2020_2021[NBA_season_2020_2021['Age'] == NBA_season_2020_2021['Age'].max()]
print(old_player)


            Player Pos  Age   Tm  G  GS   MP   FG  FGA  FG%  ...  FT%  ORB  \
279  Udonis Haslem   C   40  MIA  1   0  3.0  2.0  2.0  1.0  ...  0.0  0.0   

     DRB  TRB  AST  STL  BLK  TOV   PF  PTS  
279  1.0  1.0  0.0  0.0  0.0  0.0  0.0  4.0  

[1 rows x 29 columns]


We can't use this as he does not have many games played lets look at someone with at least 20 minutes played to get better results.

In [31]:
# Get the oldest player with at least 20 minutes played per game
old_player = NBA_season_2020_2021[NBA_season_2020_2021['MP'] >= 20.0].sort_values('Age', ascending=False).head(1)
print(old_player)

             Player Pos  Age   Tm   G  GS    MP   FG  FGA    FG%  ...    FT%  \
313  Andre Iguodala  SF   37  MIA  63   5  21.3  1.5  3.9  0.383  ...  0.658   

     ORB  DRB  TRB  AST  STL  BLK  TOV   PF  PTS  
313  0.6  2.9  3.5  2.3  0.9  0.6  1.1  1.4  4.4  

[1 rows x 29 columns]


In [33]:
youngest_player = NBA_season_2020_2021[NBA_season_2020_2021['MP'] >= 20.0].sort_values('Age', ascending=True).head(1)
print(youngest_player)

         Player Pos  Age   Tm   G  GS    MP   FG   FGA    FG%  ...    FT%  \
33  LaMelo Ball  PG   19  CHO  51  31  28.8  5.7  13.2  0.436  ...  0.758   

    ORB  DRB  TRB  AST  STL  BLK  TOV   PF   PTS  
33  1.2  4.7  5.9  6.1  1.6  0.4  2.8  2.7  15.7  

[1 rows x 29 columns]


In [36]:
# First select the stats we want to compare
stats = ['PTS', 'AST', 'TRB']

# Let get the players name
old_player_name = old_player['Player'].values[0]
youngest_player_name = youngest_player['Player'].values[0]

# Get the stats for each player
old_plaer_stats = old_player[stats].values.flatten()
young_player_stats = youngest_player[stats].values.flatten()

#Create the bar chart
fig = go.Figure(data=[
    go.Bar(name=old_player_name, x = stats, y = old_plaer_stats),
    go.Bar(name=youngest_player_name, x = stats, y = young_player_stats)
])

# Group the bar mode
fig.update_layout(barmode='group')

# Show the figure
fig.show()

Just by looking at the graph we can see that a younger player has a better chance of having better stats then someone who is older but we can still see that there is some difference in the amount of time played and games.

We are going to look at the old and youngest player's in that play the same position as each other and compare it that to see if we can see any difference between the two. 

In [43]:
# Get data for youngest and oldest PG
youngest_pg = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'PG') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=True).head(1)
oldest_pg = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'PG') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=False).head(1)

# Get data for the youngest and oldest SG
youngest_sg = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'SG') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=True).head(1)
oldest_sg = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'SG') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=False).head(1)

# Get data for youngest and oldest SF
youngest_sf = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'SF') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=True).head(1)
oldest_sf = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'SF') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=False).head(1)

# Get the data for the youngest and oldest PF
youngest_pf = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'PF') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=True).head(1)
oldest_pf = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'PF') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=False).head(1)

# Get the data for the youngest and oldest C
youngest_c = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'C') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=True).head(1)
oldest_c = NBA_season_2022_2023[(NBA_season_2022_2023['Pos'] == 'C') & (NBA_season_2022_2023['MP'] >= 20.0)].sort_values('Age', ascending=False).head(1)    



In [47]:
# Positions
positions = ['PG', 'SG', 'SF', 'PF', 'C']

# Ages and name of the youngest players
youngest_names = [youngest_pg['Player'].values[0], youngest_sg['Player'].values[0], youngest_sf['Player'].values[0], youngest_pf['Player'].values[0], youngest_c['Player'].values[0]]
youngest_ages = [youngest_pg['Age'].values[0], youngest_sg['Age'].values[0], youngest_sf['Age'].values[0], youngest_pf['Age'].values[0], youngest_c['Age'].values[0]]

# Ages and name of the oldest players
oldest_names = [oldest_pg['Player'].values[0], oldest_sg['Player'].values[0], oldest_sf['Player'].values[0], oldest_pf['Player'].values[0], oldest_c['Player'].values[0]]
oldest_ages = [oldest_pg['Age'].values[0], oldest_sg['Age'].values[0], oldest_sf['Age'].values[0], oldest_pf['Age'].values[0], oldest_c['Age'].values[0]]

# Create the bar chart
fig = go.Figure(data=[
    go.Bar(name='Youngest', x=positions, y=youngest_ages, text=youngest_names, textposition='auto'),
    go.Bar(name='Oldest', x=positions, y=oldest_ages, text=oldest_names, textposition='auto')
])

# Change the bar mode
fig.update_layout(barmode='group')

# Show the figure
fig.show()

Here we just wanted to see the age difference in the oldest player vs the youngest player in each position. Now lets look at how the stats compare for each player. 

In [67]:
# Create a DataFrame for the youngest and oldest players
players_df = pd.DataFrame({
    'Position': ['PG', 'SG', 'SF', 'PF', 'C', 'PG', 'SG', 'SF', 'PF', 'C'],
    'Player': [youngest_pg['Player'].values[0], youngest_sg['Player'].values[0], youngest_sf['Player'].values[0], youngest_pf['Player'].values[0], youngest_c['Player'].values[0], oldest_pg['Player'].values[0], oldest_sg['Player'].values[0], oldest_sf['Player'].values[0], oldest_pf['Player'].values[0], oldest_c['Player'].values[0]],
    'Age': [youngest_pg['Age'].values[0], youngest_sg['Age'].values[0], youngest_sf['Age'].values[0], youngest_pf['Age'].values[0], youngest_c['Age'].values[0], oldest_pg['Age'].values[0], oldest_sg['Age'].values[0], oldest_sf['Age'].values[0], oldest_pf['Age'].values[0], oldest_c['Age'].values[0]],
    'PTS': [youngest_pg['PTS'].values[0], youngest_sg['PTS'].values[0], youngest_sf['PTS'].values[0], youngest_pf['PTS'].values[0], youngest_c['PTS'].values[0], oldest_pg['PTS'].values[0], oldest_sg['PTS'].values[0], oldest_sf['PTS'].values[0], oldest_pf['PTS'].values[0], oldest_c['PTS'].values[0]],
    'AST': [youngest_pg['AST'].values[0], youngest_sg['AST'].values[0], youngest_sf['AST'].values[0], youngest_pf['AST'].values[0], youngest_c['AST'].values[0], oldest_pg['AST'].values[0], oldest_sg['AST'].values[0], oldest_sf['AST'].values[0], oldest_pf['AST'].values[0], oldest_c['AST'].values[0]],
    'TRB': [youngest_pg['TRB'].values[0], youngest_sg['TRB'].values[0], youngest_sf['TRB'].values[0], youngest_pf['TRB'].values[0], youngest_c['TRB'].values[0], oldest_pg['TRB'].values[0], oldest_sg['TRB'].values[0], oldest_sf['TRB'].values[0], oldest_pf['TRB'].values[0], oldest_c['TRB'].values[0]],
    'Type': ['Youngest', 'Youngest', 'Youngest', 'Youngest', 'Youngest', 'Oldest', 'Oldest', 'Oldest', 'Oldest', 'Oldest']
})

# Create a bar chart for the players
fig = go.Figure()

# Add traces for each stat
fig.add_trace(go.Bar(name='Age', x=players_df['Player'], y=players_df['Age'], marker_color=players_df['Type'].map({'Youngest': 'blue', 'Oldest': 'orange'})))
fig.add_trace(go.Bar(name='PTS', x=players_df['Player'], y=players_df['PTS'], marker_color=players_df['Type'].map({'Youngest': 'blue', 'Oldest': 'orange'})))
fig.add_trace(go.Bar(name='AST', x=players_df['Player'], y=players_df['AST'], marker_color=players_df['Type'].map({'Youngest': 'blue', 'Oldest': 'orange'})))
fig.add_trace(go.Bar(name='TRB', x=players_df['Player'], y=players_df['TRB'], marker_color=players_df['Type'].map({'Youngest': 'blue', 'Oldest': 'orange'})))

# Change the bar mode
fig.update_layout(barmode='group', title_text='Stats for Youngest and Oldest Players')

# Show the figure
fig.show()

Just by looking at the graph we can't really see if age difference has an impact of the player stats for each position, as we can see that some older players perform better and some younger players perform better. 